In [1]:
import ast
import logging
import os
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

## Define Agent Class

In [177]:
os.environ['OPENAI_API_KEY'] = "#######"
os.environ['OPENAI_API_TYPE'] = "azure"
os.environ["AZURE_OPENAI_ENDPOINT"] = "#######"

class ParseAgent():
    """
    I want this agent to be an expert at parsing code in a systematic way. 
    The agent could understand the main logic of the code, and helper logic. 
    The agent should be able to understand control flow. 
    """
    def __init__(self, **config):
        self._logger = logging.getLogger(f"{__name__}.{self.__class__.__name__}")
        self.llm = AzureChatOpenAI(
            openai_api_version=config["OPEN_API_VERSION"],
            azure_deployment= config["azure_deployment"],
            temperature=0
        )
        self.output_parser = StrOutputParser()

    def extract_business_logic_update(self, context):
        template_string = """Given the provided code, extract the main business logic and helper logic into separate segments.
        Save the main logic into a value of a dictionary with the key 'main_logic'.
        Save the helper logic into a value of the same dictionary with the key 'helper_logic'.
        Include the entirety of the code in either the 'main_logic' or 'helper_logic'.
        
        Code:
        ```{code}```

        Return the dictionary without additional explanations.
        """
        prompt = ChatPromptTemplate.from_template(template_string)


        extract_logic_chain = (
            {"code": RunnablePassthrough()}
            | prompt
            | self.llm
            | self.output_parser
        )

        llm_response = extract_logic_chain.invoke(context)
        try:
            extracted_code = ast.literal_eval(llm_response)
        except Exception as e:
            self._logger.error(f"LLM response not valid.")
            return llm_response
        return extracted_code, llm_response

    def organize_logic(self, context):
        template_string = """You are provided with a string representation of a dictionary where the code is separated by the main logic and helper logic. Organize this content into a dictionary.
        that breaks down each logic section into function definitions. If a function refers to another function add a key for "references" which can be a list of other functions that a function or logic references.

        ```
        Code:
        *************************
        {code}
        *************************
        
        Do not generate extra explainations, simply return the dictionary.
        """
        prompt = ChatPromptTemplate.from_template(template_string)


        organize_logic_chain = (
            {"code": RunnablePassthrough()}
            | prompt
            | self.llm
            | self.output_parser
        )

        llm_response = organize_logic_chain.invoke(context)
        try:
            extracted_code = ast.literal_eval(llm_response)
        except Exception as e:
            self._logger.error(f"LLM response not valid.")
            return llm_response
        return extracted_code

    def get_control_flow(self, context):
        template_string = """You are provided with a string representation of code. Output the control flow of the code. Include variables, function names, object names, and inputs.
        ```
        Code:
        *************************
        {code}
        *************************
        
        Do not generate extra explainations, simply return the text.
        """
        prompt = ChatPromptTemplate.from_template(template_string)


        control_flow_chain = (
            {"code": RunnablePassthrough()}
            | prompt
            | self.llm
            | self.output_parser
        )

        llm_response = control_flow_chain.invoke(context)
        return llm_response

    def generate_workflow(self, context):
        template_string = """
        I would like you to generate a text representation of a CSV file for a business process that are in raw code.
        Your first step would be to understand the input and find detailed information for NAME, ID, DESCRIPTION, TYPE, CONNECTIONS, and EXPRESSIONS based on the user's input process and sub-processes. 
        
        The output should strictly adhere to CSV format conventions without unnecessary blanks between values and contain NAME, ID, DESCRIPTION, TYPE, CONNECTIONS, and EXPRESSIONS.
        
        Follow these guidelines:
        1. 'NAME': Workflow name.
        2. 'TYPE': Workflow type, with specific rules for 'end', 'choice', and 'unknown'.
        3. 'CONNECTIONS': Flows between nodes. 
        4. 'EXPRESSIONS': Conditions or decisions at gateways.Include the ID to indicate to which node the connection flows.
        5. 'DESCRIPTION': Description of the workflow, enclosed in double quotes.
        
        Lastly, for 'type' please choose from the following available node types: 
        1. Choose 'end' if a node corresponds to an end of a task or subtask. 
        2. Choose 'choice' if a node corresponds to a decision gateway or if a task has to undergo approval.'choice' nodes should have at least two options. Be sure to capture this in 'connections' and 'expressions'. If the other option is unclear, default the node type as "unknown" and provide a plausible "name", "id", "description".

        3. Choose 'unknown' if you are unsure. Don't generate other types.
        If "choice" node has multiple references (options), ensure all subflows (starting from 'choice' node) ends with the an "end" node. Depending upon "Choice" node and condition, you may see more than one 'end' 	node. ENSURE, all 'end' nodes has its respective parent node reference.
        
        Here is the code:
         ```
        Code:
        *************************
        {code}
        *************************
        """

        prompt = ChatPromptTemplate.from_template(template_string)

        generate_flow_chain = (
            {"code": RunnablePassthrough()}
            | prompt
            | self.llm
            | self.output_parser
        )

        llm_response = generate_flow_chain.invoke(context)
        return llm_response

## Read Code

In [105]:
# Reading the file
with open('Program.cs', 'r') as file:
    code=file.read()

print(code)

using System;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text.Json;
using System.Threading.Tasks;

namespace GrantApprovalProcess
{
    // Role definition
    public enum UserRole
    {
        Administrator,
        Reviewer,
        Applicant
    }

    // Domain Object definition
    public class User
    {
        public string Name { get; set; }
        public UserRole Role { get; set; }

        public User(string name, UserRole role)
        {
            Name = name;
            Role = role;
        }
    }

    // Domain Object definition
    public class GrantApplication
    {
        public int ApplicationId { get; set; }
        public string ApplicantName { get; set; }
        public decimal RequestedAmount { get; set; }
        public bool IsNonProfit { get; set; }
    }

    // Domain Object definition
    public class GrantDecision
    {
        public int ApplicationId { get; set; }
        public bool Approved { get; set; }
        public strin

## Instanciate Class

In [179]:
config = {"OPEN_API_VERSION":"2023-07-01-preview","azure_deployment":"GPT-4-32K"}
agent  = ParseAgent(**config)

### Extract Logic

In [149]:
extracted_code, llm_response = agent.extract_business_logic_update(context=code)

In [115]:
# test = ast.literal_eval(extracted_code)
print(extracted_code['helper_logic'])


public enum UserRole
    {
        Administrator,
        Reviewer,
        Applicant
    }

    public class User
    {
        public string Name { get; set; }
        public UserRole Role { get; set; }

        public User(string name, UserRole role)
        {
            Name = name;
            Role = role;
        }
    }

    public class GrantApplication
    {
        public int ApplicationId { get; set; }
        public string ApplicantName { get; set; }
        public decimal RequestedAmount { get; set; }
        public bool IsNonProfit { get; set; }
    }

    public class GrantDecision
    {
        public int ApplicationId { get; set; }
        public bool Approved { get; set; }
        public string DecisionNote { get; set; }
    }

    public class InvalidGrantApplicationException : Exception
    {
        public InvalidGrantApplicationException(string message) : base(message)
        {
        }
    }

    public class GrantApprovalManager
    {
        private readonly

In [13]:
print(extracted_code)

{'main_logic': '\nclass Program\n{\n    static async Task Main(string[] args)\n    {\n        // Example of user roles\n        var adminUser = new User("Admin", UserRole.Administrator);\n\n        // Initialize manager with an administrator user\n        GrantApprovalManager manager = new GrantApprovalManager(adminUser);\n\n        // Create a grant application\n        var application = new GrantApplication\n        {\n            ApplicationId = 1,\n            ApplicantName = "NonProfit Org",\n            RequestedAmount = 9500,\n            IsNonProfit = true\n        };\n\n        // Evaluate the application\n        try\n        {\n            var decision = manager.EvaluateApplication(application);\n            Console.WriteLine($"Decision for Application {decision.ApplicationId}: {decision.DecisionNote}");\n\n            // Fetch a suggested activity\n            await manager.FetchBoredActivityAsync();\n        }\n        catch (InvalidGrantApplicationException e)\n        {\

### Organize Logic

In [151]:
organized = agent.organize_logic(llm_response)

In [27]:
organized = ast.literal_eval(organized)

In [153]:
organized.keys()

dict_keys(['main_logic', 'helper_logic'])

In [161]:
print(organized['helper_logic']['GrantApprovalManager']['code'])

public class GrantApprovalManager
    {
        private readonly HttpClient httpClient = new HttpClient();
        private readonly string boredApiUrl = "https://www.boredapi.com/api/activity";
        private User currentUser;

        public GrantApprovalManager(User user)
        {
            currentUser = user;
            var simulatedApiKey = Environment.GetEnvironmentVariable("SIMULATED_API_KEY");
            if (!string.IsNullOrEmpty(simulatedApiKey))
            {
                httpClient.DefaultRequestHeaders.Add("X-API-KEY", simulatedApiKey);
            }
        }
        public GrantDecision EvaluateApplication(GrantApplication application)
        {
            if (currentUser.Role != UserRole.Administrator && currentUser.Role != UserRole.Reviewer)
            {
                throw new InvalidOperationException("Insufficient permissions to evaluate applications.");
            }

            var decision = new GrantDecision { ApplicationId = application.ApplicationI

In [139]:
print(organized['helper_logic'].keys())

dict_keys(['function', 'code', 'references'])


In [143]:
print(organized['helper_logic']['code'])

public enum UserRole
    {
        Administrator,
        Reviewer,
        Applicant
    }

    public class User
    {
        public string Name { get; set; }
        public UserRole Role { get; set; }

        public User(string name, UserRole role)
        {
            Name = name;
            Role = role;
        }
    }

    public class GrantApplication
    {
        public int ApplicationId { get; set; }
        public string ApplicantName { get; set; }
        public decimal RequestedAmount { get; set; }
        public bool IsNonProfit { get; set; }
    }

    public class GrantDecision
    {
        public int ApplicationId { get; set; }
        public bool Approved { get; set; }
        public string DecisionNote { get; set; }
    }

    public class InvalidGrantApplicationException : Exception
    {
        public InvalidGrantApplicationException(string message) : base(message)
        {
        }
    }

    public class GrantApprovalManager
    {
        private readonly

### Get Control flow

In [101]:
print(organized['helper_logic'][1]['code'])
# test = organized['helper_logic']['GrantApprovalManager']['code']

public class GrantApplication
    {
        public int ApplicationId { get; set; }
        public string ApplicantName { get; set; }
        public decimal RequestedAmount { get; set; }
        public bool IsNonProfit { get; set; }
    }


control_flow = agent.get_control_flow(test)

## Generate Workflow

In [181]:
test_output = agent.generate_workflow(organized['helper_logic']['GrantApprovalManager']['code'])

In [183]:
print(test_output)

"NAME","ID","DESCRIPTION","TYPE","CONNECTIONS","EXPRESSIONS"
"GrantApprovalManager","1","Class that manages the grant approval process","unknown","",""
"GrantApprovalManager Constructor","2","Initializes the GrantApprovalManager with a user and sets up the HttpClient","unknown","1",""
"EvaluateApplication","3","Evaluates a grant application based on the user's role and the application's requested amount and non-profit status","choice","1","currentUser.Role == UserRole.Administrator || currentUser.Role == UserRole.Reviewer"
"FetchBoredActivityAsync","4","Fetches a random activity from the Bored API","unknown","1",""
"Decision: Approved","5","The grant application is approved","end","3","application.RequestedAmount <= 10000 && application.IsNonProfit"
"Decision: Rejected","6","The grant application is rejected","end","3","application.RequestedAmount > 10000 || !application.IsNonProfit"
"Error: Insufficient Permissions","7","An error is thrown if the user does not have sufficient permissi